In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, to_date, when, date_format, month

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("CovidStats") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
df = spark.read.csv("./complete.csv", header=True, inferSchema=True)